In [1]:
import sys
import time
import argparse
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import keras as keras

from sys import argv,exit
from datetime import datetime, date, timedelta
from keras.models import Sequential
from keras.layers import Dense,Dropout,GRU, LSTM,Reshape
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error

Using TensorFlow backend.


In [2]:
def RNN_model(network_type, df,num_iterations,num_features,batch_size,hidden_layers,num_neurons,dropout_rate):

    random.seed()
    global net
    net = Sequential()
    
    experiment = experiment = str(datetime.now()).replace(":","").replace(" ","").replace("-","")[:14]
    
    learning_rate = 0.002
    #This optimizer is usually a good choice for recurrent neural networks
    #It is recommended to leave the parameters of this optimizer at their default values.
    nadam = keras.optimizers.Nadam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    optimizer = 'nadam'
   
    label = '08_close'
    date_column = '00_date'

    maxs =[]
    mins =[]

    def buildGRU(w_init="glorot_uniform",act="tanh"):
        net.add(Dense(num_features,kernel_initializer=w_init,input_dim=num_features,activation='linear'))
        net.add(Reshape((1,num_features)))
        net.add(BatchNormalization())
        for i in range(hidden_layers):
            net.add(GRU(num_neurons,kernel_initializer=w_init,activation=act,return_sequences=True))
            net.add(Dropout(dropout_rate))
        net.add(GRU(num_neurons,kernel_initializer=w_init,activation=act,return_sequences=False))
        net.add(Dropout(dropout_rate))
        net.add(Dense(1,kernel_initializer=w_init,activation='linear'))
        net.compile(optimizer=nadam,loss='mean_squared_error')
        
    def buildLSTM(w_init="glorot_uniform",act="tanh"):
        net.add(Dense(num_features,kernel_initializer=w_init,input_dim=num_features,activation='linear'))
        net.add(Reshape((1,num_features)))
        net.add(BatchNormalization())
        for i in range(hidden_layers):
            net.add(LSTM(num_neurons,kernel_initializer=w_init,activation=act,return_sequences=True))
            net.add(Dropout(dropout_rate))
        net.add(LSTM(num_neurons,kernel_initializer=w_init,activation=act,return_sequences=False))
        net.add(Dropout(dropout_rate))
        net.add(Dense(1,kernel_initializer=w_init,activation='linear'))
        net.compile(optimizer=nadam,loss='mean_squared_error')        
        
    def chart(real,predicted,xlabel,show=True):
        plt.title('Real BTC price vs Predicted BTC price')
        plt.plot(predicted,label='Predicted')
        plt.plot(real, label='Real')
        plt.ylabel('BTC/USD')
        plt.xlabel(xlabel)
        plt.savefig("BTC_chart"+experiment+".png")
        plt.grid(True)
        plt.legend()
        if show:plt.show()

    def loadData(df):
        #column names
        columns = df.columns.tolist()
        columns = [x for x in columns if x != label] # removes the label column
        columns = [x for x in columns if x != date_column] # removes the date column
        columns = columns[:num_features] #first N features
        #get the values for a given column
        Y = df[label].values.tolist()
        #get a data frame with selected columns
        X = df[columns].values.tolist()
        #for i in range(len(X)): X[i] = [float(k) for k in X[i]]
        #number of features
        #num_features = len(columns)
        return X[:-1],Y[1:] # X[i] will predict Y[i+1]

    def reduceVector(vec,getVal=False):
        vect = []
        mx,mn = max(vec),min(vec)
        mx = mx+mn
        mn = mn-((mx-mn)*0.4)
        for x in vec:
            vect.append((x-mn)/(mx-mn+sys.float_info.min))
        if not getVal:return vect
        else:return vect,mx,mn

    def reduceValue(x,mx,mn):
        return (x-mn)/(mx-mn+sys.float_info.min)

    def augmentValue(x,mx,mn):
        return (mx-mn)*x+mn

    def reduceMatRows(data):
        l = len(data[0])
        for i in range(l):
            v = []
            for t in range(len(data)):
                v.append(data[t][i])
            v,mx,mn = reduceVector(v,getVal=True)
            maxs.append(mx)
            mins.append(mn)
            for t in range(len(data)):
                data[t][i] = v[t]
        return data

    def reduceCurrent(data):
        for i in range(len(data)):
            data[i] = reduceValue(data[i],maxs[i],mins[i])
        return data
    
    def MASE(training_series, testing_series, prediction_series):
        n = training_series.shape[0]
        d = np.abs(  np.diff( training_series) ).sum()/(n-1)    
        errors = np.abs(testing_series - prediction_series )
        return errors.mean()/d    

    #data loading:
    data,Y = loadData(df)
    #data normalization
    data = reduceMatRows(data)
    labels,m1,m2 =reduceVector(Y,getVal=True)

    #Assembling Net:
    if network_type == 'GRU': buildGRU()
    else: buildLSTM()

    # Data split: training (75,5%) validation (26,5%) testing (1,9%)
    num_records = len(labels)
    num_validation = 396
    num_testing = 31
    num_training = num_records-num_testing

    training_X = data[:num_training]
    training_Y = labels[:num_training]

    testing_X = data[num_training:]
    testing_Y = labels[num_training:]

    #Training GRU
    startTime = datetime.now()
    with tf.device("/gpu:0"): net.fit(training_X,training_Y,epochs=num_iterations,batch_size=batch_size)
    #with tf.device("/gpu:0"): net.fit(training_X,training_Y,epochs=10,batch_size=batch_size)
    print("Training complete: "+experiment,end="\n")
    #net.save_weights(network_type+"_model_"+experiment+".h5")
    #print("Model saved as "+network_type+"_model_"+experiment+".h5")
    training_time = datetime.now() - startTime
    print("Training time:", training_time)

    ### Predict all over the dataset to build the chart
    reals,preds = [],[]
    startTime = datetime.now()
    length = len(testing_Y)
    for i in range(0,length):
        x = np.array(testing_X[i]).reshape(1,num_features)
        predicted = augmentValue(net.predict(x)[0],m1,m2)[0]
        real = augmentValue(testing_Y[i],m1,m2)
        preds.append(predicted)
        reals.append(real)

    RMSE_T = np.sqrt(mean_squared_error(reals, preds))
    RMSE_D = np.sqrt(mean_squared_error(reals[:1], preds[:1]))
    RMSE_W = np.sqrt(mean_squared_error(reals[:7], preds[:7]))
    
    # PLOTTING
    chart(reals,preds,str(num_testing)+" days (from September 1st, 2017)")
    chart(Y[:num_training]+reals,Y[:num_training]+preds,str(num_records)+" days (from May 2nd, 2013)")
    chart((Y[:num_training]+reals)[num_training-10:],(Y[:num_training]+preds)[num_training-10:],"zoom")

    print ("Root Mean Squared Error (RMSE): %f" % RMSE_T)
    print ("RMSE for the next day (1 day): %f" % RMSE_D)
    print ("RMSE for the next week (7 days): %f" % RMSE_W)

    print("Num. iterations: "+str(num_iterations),end="\n")
    print("Batch size: "+str(batch_size),end="\n")
    print("Optimizer: "+optimizer,end="\n")
    print("Hidden layers: "+str(hidden_layers),end="\n")
    print("Dropout rate: "+str(dropout_rate),end="\n")
    print("Learning rate: "+str(learning_rate),end="\n\n")

    dates = []
    for i in range(31): dates+=[date(2017,9,1)+timedelta(days=i)]    
    predictions = pandas.DataFrame({'date': dates, 'real': reals, 'predictions': preds })
    print(predictions)
    
    results_columns = ['experiment','num_iterations','num_features','batch_size','hidden_layers','neurons','dropout','training_time', 'RMSE','RMSE_1D', 'RMSE_1W']
    results = [[experiment,num_iterations,num_features,batch_size,hidden_layers,num_neurons,dropout_rate,training_time,RMSE_T,RMSE_D,RMSE_W]]
    return pandas.DataFrame(results,columns=results_columns)


In [ ]:
# HYPER-PARAMETER OPTIMIZATION
file_name = 'btc_multivariate.xlsx'
df = pandas.read_excel(file_name)

network_type = 'GRU'

dr = RNN_model(df,50,11,100,1,10,0.1)
for i in [100]: #iterations
    for f in [11,75,213]: #features
        for b in [100,300,500]:#batch_size
            for l in [1,3,10]: #num_layers
                for n in [10,100,500]: #num_neurons
                    for d in [0.1,0.3]: #dropout_rate
                        frames = [dr,RNN_model(network_type,df,i,f,b,l,n,d)]
                        dr = pandas.concat(frames)
            writer = pandas.ExcelWriter(network_type+'_output_'+str(i)+'_'+str(f)+'_'+str(b)+'.xlsx')
            dr.to_excel(writer,'Sheet1')
            writer.save()

In [ ]:
# HYPER-PARAMETER OPTIMIZATION
file_name = 'btc_multivariate.xlsx'
df = pandas.read_excel(file_name)

network_type = 'LSTM'

dr = RNN_model(df,50,11,100,1,10,0.1)
for i in [100]: #iterations
    for f in [11,75,213]: #features
        for b in [100,300,500]:#batch_size
            for l in [1,3,10]: #num_layers
                for n in [10,100,500]: #num_neurons
                    for d in [0.1,0.3]: #dropout_rate
                        frames = [dr,RNN_model(network_type,df,i,f,b,l,n,d)]
                        dr = pandas.concat(frames)
            writer = pandas.ExcelWriter(network_type+'_output_'+str(i)+'_'+str(f)+'_'+str(b)+'.xlsx')
            dr.to_excel(writer,'Sheet1')
            writer.save()